In [1]:
import numpy as np
import pandas as pd
from statsmodels.api import OLS, add_constant

In [ ]:
xls = pd.ExcelFile('./bank_datastream0816.xlsx')
for sheet_name in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name=sheet_name)
    df.to_csv(f'{sheet_name}.csv', index=False)


In [ ]:
stock_data=pd.read_csv("./share price daily.csv")
market_data=pd.read_csv("./market indices_daily.csv")
riskfree_data=pd.read_csv("./EONIA.csv")

In [ ]:
stock_data.head()

In [ ]:
market_data

In [ ]:
# Parameters
event_date = '2020-03-27'
estimation_window_start = -63  # Start of the estimation window
estimation_window_end = -3     # End of the estimation window
event_window_start = -3  #  days before the event
event_window_end = 3     #  days after the event
bankname = stock_data.iloc[0:0,1:]
marketindex = market_data.iloc[0:0,1:]
riskfreeindex = riskfree_data.iloc[0:0,1:]
bank_list = bankname.columns[0:].tolist()  # Convert the first row of stock_data to a list
market_list = marketindex.columns[0:].tolist() 
riskfree_list = riskfreeindex.columns[0:].tolist() 

In [ ]:
print(bank_list)

In [ ]:
print(market_list)

In [ ]:
print(riskfree_list)

In [ ]:
print("Column names:", stock_data.columns)

In [ ]:
# Clean up column names
stock_data.columns = stock_data.columns.str.strip()
market_data.columns = market_data.columns.str.strip()
riskfree_data.columns = riskfree_data.columns.str.strip()

# Convert 'Date' column to datetime
date_column_name = 'Date'
stock_data[date_column_name] = pd.to_datetime(stock_data[date_column_name])
market_data[date_column_name] = pd.to_datetime(market_data[date_column_name])
riskfree_data[date_column_name] = pd.to_datetime(riskfree_data[date_column_name])

# Set Date as index if it's not already
stock_data.set_index(date_column_name, inplace=True)
market_data.set_index(date_column_name, inplace=True)
riskfree_data.set_index(date_column_name, inplace=True)


In [ ]:
print("Columns in riskfree_data:", riskfree_data.columns)

In [ ]:
# Initialize an empty DataFrame to store daily returns
bank_daily_returns = pd.DataFrame()
market_daily_returns = pd.DataFrame()

selected_metric = 'EONIARATE'
riskfree_data_series = pd.to_numeric(riskfree_data[selected_metric], errors='coerce')
    
# Iterate over each column to calculate daily returns
for column in stock_data.columns:
    if column != date_column_name:  # Skip the Date column
        # Ensure the column data is numeric
        stock_data[column] = pd.to_numeric(stock_data[column], errors='coerce')
        
        # Calculate daily returns for this column
        bank_daily_returns[column] = stock_data[column].pct_change()
      
        # Calculate excess returns by subtracting the risk-free rate
        bank_daily_returns[column] = bank_daily_returns[column] - riskfree_data_series

# If the Date column was reset earlier, there's no need to add it back again
if date_column_name not in bank_daily_returns.columns:
    bank_daily_returns.reset_index(inplace=True)

# Reorder columns so that Date appears first
bank_daily_returns = bank_daily_returns[[date_column_name] + [col for col in bank_daily_returns.columns if col != date_column_name]]


In [ ]:
print(bank_daily_returns)

In [ ]:
market_daily_returns = pd.DataFrame(index=market_data.index)
for column in market_data.columns:
    if column != date_column_name:  # Skip the Date column
        # Ensure the column data is numeric
        market_data[column] = pd.to_numeric(market_data[column], errors='coerce')
        
        # Calculate daily returns for this column
        market_daily_returns[column] = market_data[column].pct_change()
        
        # Calculate excess returns by subtracting the risk-free rate
        market_daily_returns[column] = market_daily_returns[column] - riskfree_data_series
        
print(market_daily_returns)

In [ ]:
# Merge the two DataFrames on the Data column
merged_data=pd.merge(bank_daily_returns, market_daily_returns, on=date_column_name, suffixes=('_stock', '_market'))
print(merged_data)

In [ ]:
# Define the estimation period 
estimation_start_date = pd.to_datetime(event_date) + pd.Timedelta(days=estimation_window_start)
estimation_end_date = pd.to_datetime(event_date) + pd.Timedelta(days=estimation_window_end)

# Define the event period 
event_start_date = pd.to_datetime(event_date) + pd.Timedelta(days=event_window_start)
event_end_date = pd.to_datetime(event_date) + pd.Timedelta(days=event_window_end)

# Filter estimation data

estimation_data = merged_data[(merged_data[date_column_name] >= estimation_start_date) & (merged_data[date_column_name] <= estimation_end_date)]

# Filter event data
event_data = merged_data[(merged_data[date_column_name] >= event_start_date) & (merged_data[date_column_name]<= event_end_date)]

In [ ]:
print(estimation_data)

In [ ]:
print(estimation_data.columns)

In [ ]:
# Market model: Regress stock returns on market returns
#STOXX EUROPE 50
# Perform multivariate regression for all banks against the market indices
stoxx = market_list[0]
sp = market_list[1]


# Perform OLS regression for each bank against the market indices
results = {}
cumulative_abnormal_returns = pd.DataFrame()

for bank in bank_list:
    X = estimation_data[stoxx]  # Independent variables (market returns)
    X = add_constant(X)  # Add a constant (intercept)
    y = estimation_data[bank]  # Dependent variable (bank returns)
    
    
    model = OLS(y, X).fit()  # Fit the model
    
    print(model.summary())
    
    results = {}
    
    # Predict expected returns during the event window
    event_data[f'Expected_{bank}'] = model.predict(add_constant(event_data[stoxx]))
    
    # Calculate abnormal returns
    event_data[f'Abnormal_{bank}'] = event_data[bank] - event_data[f'Expected_{bank}']
    
    # Calculate cumulative abnormal returns (CAR)
    event_data[f'CAR_{bank}'] = event_data[f'Abnormal_{bank}'].cumsum()
    
    # Save CAR
    cumulative_abnormal_returns[f'CAR_{bank}'] = event_data[f'CAR_{bank}']
   
    # Calculate cumulative Average abnormal returns (CAAR)
    average_caar = cumulative_abnormal_returns.mean()
    

    
#     # Print the summary for each bank
#     print(f"Regression results for {bank}:")
#     print(model.summary())
#     print("\n" + "="*80 + "\n")
    
    # Display the results
print("Estimation Start Date:", estimation_start_date)
print("Estimation End Date:", estimation_end_date)


In [ ]:
print(event_data[['Date'] + [f'Abnormal_{bank}' for bank in bank_list] + 
                       [f'CAR_{bank}' for bank in bank_list]])
    

In [ ]:
pd.options.display.max_rows = None
average_caar.index = average_caar.index.str.replace('CAR_', '')
print(average_caar)

In [ ]:
event_date = pd.to_datetime('2020-03-27')
event_data['Date'] = pd.to_datetime(event_data['Date'], errors='coerce')
event_data['days_from_event'] = (event_data['Date'] - event_date).dt.days

windows = {
    'CAAR (-1,+1)': (-1, 1),
    'CAAR (-1,+3)': (-1, 3),
    'CAAR (0,+1)': (0, 1),
}
caar_results = pd.DataFrame()
# Assume event_data contains the columns: 'Bank Name', 'days_from_event', and 'Abnormal_Return'

for label, (start, end) in windows.items():
    # Filter data for the specified time window
    window_data = event_data[(event_data['days_from_event'] >= start) & (event_data['days_from_event'] <= end)]
    
    # Calculate CAAR for each bank
    caar_per_bank = {}
    for bank in bank_list:
        if bank in window_data.columns:
            caar_per_bank[bank] = window_data[f'Abnormal_{bank}'].mean()

    # Convert to DataFrame for merging
    caar_per_bank_df = pd.DataFrame(list(caar_per_bank.items()), columns=['Bank', label])
    
    # Merge the results into the caar_results DataFrame
    if caar_results.empty:
        caar_results = caar_per_bank_df
    else:
        caar_results = pd.merge(caar_results, caar_per_bank_df, on='Bank', how='outer')
        
        # Convert CAAR results to percentages and format them to two decimal places
for column in caar_results.columns[1:]:  # Skip the 'Bank' column
    caar_results[column] = caar_results[column] * 100  # Convert to percentage
    caar_results[column] = caar_results[column].apply(lambda x: f"{x:.2f}%")  # Format to two decimal places

# Display the calculated CAAR results for each company
print(caar_results)
    

In [ ]:
output_file_path = './Complete_CAAR_Results_63_stoxx_EONIA.xlsx'
caar_results.to_excel(output_file_path, index=False)

output_file_path

In [ ]:
# output_file_path = './Complete_CAAR_Results.xlsx'
# caar_results.to_excel(output_file_path, index=False)

# output_file_path